In [1]:
import json
import random
from datasets import load_dataset

In [2]:
dataset = load_dataset('Qwen/ProcessBench', split='olympiadbench')

In [12]:
with open("./resources/prompt_templates/DEPENDENCY_TRACKING_1.txt") as f:
    DEPENDENCY_TRACKING_PROMPT = f.read()

In [13]:
sample_id = random.choice(range(len(dataset)))
print(f"Sample Id: {sample_id}")
print("="*10)
print(dataset[sample_id]["problem"])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_response = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"])])
print(DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_response=tagged_response
))

Sample Id: 201
Suppose that $a(x+b(x+3))=2(x+6)$ for all values of $x$. Determine $a$ and $b$.
You are an expert reasoning graph architect. Given a problem and a step-by-step solution, identify the **direct dependencies** between steps. For each step `t` (0-indexed), output a list of step indices that step `t` **directly requires** to justify its correctness. Adhere strictly to these rules:

1. **Direct Use Only**:  
   Include step `i` in step `t`'s dependencies **only if** step `t` explicitly uses the result, lemma, or output from step `i`. Ignore indirect dependencies.

2. **Minimal & Non-Redundant**:  
   - If step `t` uses identical information from multiple steps (e.g., the same lemma), include **only the earliest step** (smallest index).  
   - The set must be the **minimal** required to verify step `t`.

3. **Output Format**:  
   - Return a JSON object with keys as step indices (integers) and values as sorted lists (ascending order).  
   - Step `0` depends on nothing: its lis

In [9]:
sample_id = random.choice(range(len(dataset)))
step_id = random.choice(range(1, len(dataset[sample_id]["steps"])))
print(f"Sample Id: {sample_id}")
print(f"Step Id: {step_id}")
print("="*10)
print(dataset[sample_id]["problem"])
print("="*10)
print(dataset[sample_id]["steps"][step_id])
print("="*10)
problem = dataset[sample_id]["problem"]
tagged_previous_steps = "\n".join([f"[{i}] {step}" for i, step in enumerate(dataset[sample_id]["steps"][:step_id])])
tagged_current_step = f"[{step_id}] {dataset[sample_id]['steps'][step_id]}"
print(DEPENDENCY_TRACKING_PROMPT.format(
    problem=problem, tagged_previous_steps=tagged_previous_steps, tagged_current_step=tagged_current_step
))

Sample Id: 527
Step Id: 7
Square $K E N T$ has side length 20 . Point $M$ lies in the interior of $K E N T$ such that $\triangle M E N$ is equilateral. Given that $K M^{2}=a-b \sqrt{3}$, where $a$ and $b$ are integers, compute $b$.
Substituting the value of \( x^2 \) from the previous step, we get \( KM^2 = 20^2 + \frac{1600 - 1200\sqrt{3}}{3} \).
You are an expert in logical dependency analysis for step-by-step problem solving.

**Task: **
Given a problem statement, a current step t, and a window of k previous steps, identify the **direct dependencies** of step t by outputting the indexes of the previous steps that step t **explicitly and necessarily relies on**. Only include steps whose results or lemmas are directly used in step t such that, if those steps are correct, step t can be verified independently.

**Input Format:**
- **Problem:** [Problem description]
- **Previous Steps:**
  - [index-1] [Step description]
  . . .
  - [index-k] [Step description]
- **Current Step:**
  - [in

In [8]:
# sample_id = random.choice(range(len(dataset)))
# step_id = random.choice(range(len(dataset[sample_id]["steps"])))
# print(f"Sample Id: {sample_id}")
# print(f"Step Id: {step_id}")
# print("="*10)
# print(dataset[sample_id]["problem"])
# print("="*10)
# print(dataset[sample_id]["steps"][step_id])
# print("="*10)
# print(STATE_EXTRACTING_PROMPT.format(problem=dataset[sample_id]["problem"], step=dataset[sample_id]["steps"][step_id]))

In [33]:
print(STATE_EXTRACTING_PROMPT.format(problem=dataset[502]["problem"], step="\n".join([f"({i}) {step}" for i, step in enumerate(dataset[502]["steps"])])))

You are given one reasoning step from a mathematical solution. Your job is to read exactly that step, ignore everything else, and pull out *only* the new fact(s), result(s), or object(s) that this step *adds* to the proof. We call each such new piece of information a “state.” Don’t worry about labeling or classifying the state—just list each one as a separate bullet point, in natural mathematical language.

A “state” is any of:
- A numeric or symbolic expression newly computed (e.g. “DC = 24/5,” “x = (–b ± √(b²–4ac))/(2a)”).
- A new object or construction introduced (e.g. “Let D be the foot of the perpendicular from B to AC”).
- A relation or constraint established (e.g. “AB ⟂ DC,” “x ≠ 0,” “n is even”).
- A mini‐lemma or formula invocation (e.g. “Area(ΔABC) = ½·base·height,” “By Cauchy–Schwarz, …”).

**Do NOT** include:
- Justification or proof details beyond a very short phrase.
- Anything that isn’t a concrete fact used later.

---

**Example**  
Problem:  
In triangle \(ABC\), \(AB

In [ ]:
[
    "f(3) = 15",
    "f(6) = 2f(1.5) + 3",
    "f(6) = 33"
]

In [23]:
for id, (a, b) in enumerate(zip(range(10), None or [None]*10)):
    print(id, a, b)



0 0 None
1 1 None
2 2 None
3 3 None
4 4 None
5 5 None
6 6 None
7 7 None
8 8 None
9 9 None


In [25]:
None or set()

set()

In [36]:
tmp = [[1,2,3,4], [3,4,5,6], [3,4]]
idx = [i for i, prompt_list in enumerate(tmp) for _ in range(len(prompt_list))]
prompts = [prompt for prompt_list in prompts for prompt in prompt_list]


In [37]:
idx

[0, 0, 0, 0, 1, 1, 1, 1, 2, 2]